# GA4GH IPython Example Notebook

This notebook provides an overview of how to call a the GA4GH reference server from an iPython notebook.  Before running this notebook:

1. git clone https://github.com/ga4gh/server.git -b develop
2. Download the example data (if $scripts/download\_data.py$ doens't work, wget http://www.well.ox.ac.uk/~jk/ga4gh-example-data-v3.0.tar and tar -xvf in the server/ directory
3. Launch an instance of the reference server on localhost ("python server_dev.py")

## Connect to GA4GH Server

In [1]:
import ga4gh.client
baseURL = "http://localhost:8000"
client = ga4gh.client.HttpClient(baseURL)

Great! Now we can run calls to query ReferenceSets, References, Datasets, VariantSets, CallSets, Variants, ReadGroups, ReadGroupSets, & Reads

## Search/Get ReferenceSets

ReferenceSets are lists of References.  Think of them like the genome version (ie. grch37, hg19).  This NCBI Taxon Id for our human dataset is:

In [2]:
results = client.searchReferenceSets()
for result in results:
    print result.ncbiTaxonId
    referenceSetId = result.id

9606


In [3]:
referenceSet = client.getReferenceSet(referenceSetId)
print referenceSet

ReferenceSet({"name": null, "sourceURI": "TODO", "assemblyId": "TODO", "sourceAccessions": [], "ncbiTaxonId": 9606, "isDerived": false, "id": "R1JDaDM4LXN1YnNldA==", "md5checksum": "234ea63052f999c21c2bdb6f60e61038", "description": "TODO"})


## Search/Get References 

The References endpoints let you access DNA sequences. The example dataset is from subsets of the first 3 chromosomes:

In [4]:
results = client.searchReferences(referenceSetId)
for result in results:
    print result.name, result.length
    referenceId = result.id

1 138395
2 32403
3 81617


In [5]:
reference = client.getReference(referenceId)
print reference

Reference({"name": "3", "sourceURI": "http://www.ebi.ac.uk/ena/data/view/CM000665.2%26range=0-81617&display=fasta", "sourceAccessions": ["CM000665.2.subset"], "sourceDivergence": null, "length": 81617, "md5checksum": "0680724fdd32e4dba0c14db5cef24f1e", "isDerived": false, "id": "R1JDaDM4LXN1YnNldDoz", "ncbiTaxonId": 9606})


In addition to fetching metadata about the reference, you can access the base sequence:

In [6]:
client.listReferenceBases(referenceId, start=10000, end=10100)

u'CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTCACCCTACCCTAACCCTAACCCTAACCCTCACCCTCACCCTCACTCAACC'

## Search/Get Dataset

In [7]:
results = client.searchDatasets()
for result in results:
    print result.name
    datasetId =  result.id

1kg-p3-subset


In [8]:
dataset = client.getDataset(datasetId)
print dataset

Dataset({"description": null, "name": "1kg-p3-subset", "id": "MWtnLXAzLXN1YnNldA=="})


## Search/Get VariantSets

In [9]:
results = client.searchVariantSets(datasetId)
for result in results:
    print result.name
    variantSetId = result.id

mvncall


In [10]:
variantSet = client.getVariantSet(variantSetId)
print variantSet

VariantSet({"name": "mvncall", "referenceSetId": "", "id": "MWtnLXAzLXN1YnNldDptdm5jYWxs", "datasetId": "MWtnLXAzLXN1YnNldA==", "metadata": [{"info": {}, "description": "", "number": "1", "value": "VCFv4.1", "key": "version", "type": "String", "id": ""}, {"info": {}, "description": "Confidence interval around END for imprecise variants", "number": "2", "value": "", "key": "INFO.CIEND", "type": "Integer", "id": ""}, {"info": {}, "description": "Confidence interval around POS for imprecise variants", "number": "2", "value": "", "key": "INFO.CIPOS", "type": "Integer", "id": ""}, {"info": {}, "description": "Source call set.", "number": "1", "value": "", "key": "INFO.CS", "type": "String", "id": ""}, {"info": {}, "description": "End coordinate of this variant", "number": "1", "value": "", "key": "INFO.END", "type": "Integer", "id": ""}, {"info": {}, "description": "Imprecise structural variation", "number": "0", "value": "", "key": "INFO.IMPRECISE", "type": "Flag", "id": ""}, {"info": {}, 

## Search/Get Callset

Callsets apply to the samples within a dataset

In [11]:
results = client.searchCallSets(variantSetId)
for result in results:
    print result.name
    callSetId =  result.id

HG00096
HG00533
HG00534


In [12]:
callset = client.getCallSet(callSetId)
print callset

CallSet({"info": {}, "updated": 1444663564000, "name": "HG00534", "created": 1444663564000, "sampleId": "HG00534", "variantSetIds": ["MWtnLXAzLXN1YnNldDptdm5jYWxs"], "id": "MWtnLXAzLXN1YnNldDptdm5jYWxsOkhHMDA1MzQ="})


## Search/Get Variants

In [44]:
results = client.searchVariants(variantSetId, referenceName = "1")
for result in results:
    print result.referenceName, result.start, result.names[0]
    variantId = result.id

1 10176 rs367896724
1 10234 rs540431307
1 10351 rs555500075
1 10504 rs548419688
1 10505 rs568405545
1 10510 rs534229142
1 10538 rs537182016
1 10541 rs572818783
1 10578 rs538322974
1 10615 rs376342519
1 10641 rs558604819
1 11007 rs575272151
1 11011 rs544419019
1 11062 rs561109771
1 13010 rs574746232
1 13109 rs540538026
1 13115 rs62635286
1 13117 rs200579949
1 13155 rs552314247
1 13258 rs562993331
1 13272 rs531730856
1 13283 rs548333521
1 13288 rs568318295
1 13288 rs538791886
1 13312 rs527952245
1 13364 rs548087592
1 13379 rs571093408
1 13381 rs538606945
1 13444 rs558318514
1 13452 rs568927457
1 13481 rs537951473
1 13482 rs554760071
1 13493 rs574697788
1 13542 rs540466151
1 13549 rs554008981
1 14461 rs577106641
1 14463 rs546169444
1 14563 rs562748080
1 14598 rs531646671
1 14603 rs541940975
1 14673 rs561913721
1 14718 rs527865771
1 14727 rs547701710
1 14774 rs571121669
1 14859 rs533499096
1 14873 rs552113149
1 14929 rs75454623
1 14932 rs199856693
1 14974 rs554686622
1 15030 rs568188357
1 

In [48]:
variant = client.getVariant(variantId)
print variant

Variant({"info": {"EUR_AF": ["0.00889999978244"], "SAS_AF": ["0.00410000002012"], "AC": ["0"], "AA": [".|||"], "AF": ["0.00319488998502"], "AFR_AF": ["0.00079999997979"], "AMR_AF": ["0.00289999996312"], "AN": ["6"], "VT": ["SNP"], "EAS_AF": ["0.0"], "NS": ["2504"], "DP": ["28809"]}, "updated": 1444663564000, "end": 138396, "calls": [{"info": {}, "genotype": [0, 0], "callSetId": "MWtnLXAzLXN1YnNldDptdm5jYWxsOkhHMDAwOTY=", "phaseset": "*", "genotypeLikelihood": [], "callSetName": "HG00096"}, {"info": {}, "genotype": [0, 0], "callSetId": "MWtnLXAzLXN1YnNldDptdm5jYWxsOkhHMDA1MzM=", "phaseset": "*", "genotypeLikelihood": [], "callSetName": "HG00533"}, {"info": {}, "genotype": [0, 0], "callSetId": "MWtnLXAzLXN1YnNldDptdm5jYWxsOkhHMDA1MzQ=", "phaseset": "*", "genotypeLikelihood": [], "callSetName": "HG00534"}], "created": 1444663564000, "variantSetId": "MWtnLXAzLXN1YnNldDptdm5jYWxs", "referenceBases": "G", "start": 138395, "names": ["rs568894656"], "alternateBases": ["A"], "referenceName": "1

## Search/Get Readgroupsets

In [54]:
results = client.searchReadGroupSets(datasetId)
for result in results:
    print result.name
    readGroupSetId = result.id
    readGroupId = result.readGroups[0].id

HG00096
HG00533
HG00534


In [55]:
readgroupset = client.getReadGroupSet(readGroupSetId)
print readgroupset

ReadGroupSet({"readGroups": [{"info": {}, "updated": 1444697504850, "predictedInsertSize": 486, "stats": {"unalignedReadCount": -1, "alignedReadCount": -1, "baseCount": null}, "description": "SRP001293", "created": 1444697504850, "programs": [{"commandLine": "bwa index -a bwtsw $reference_fasta", "prevProgramId": null, "id": "bwa_index", "version": "0.5.9-r16", "name": "bwa"}, {"commandLine": "bwa aln -q 15 -f $sai_file $reference_fasta $fastq_file\tPP:bwa_index", "prevProgramId": null, "id": "bwa_aln_fastq", "version": "0.5.9-r16", "name": "bwa"}, {"commandLine": "bwa sampe -a 1458 -r $rg_line -f $sam_file $reference_fasta $sai_file(s) $fastq_file(s)\tPP:bwa_aln_fastq", "prevProgramId": null, "id": "bwa_sam", "version": "0.5.9-r16", "name": "bwa"}, {"commandLine": "samtools view -bSu $sam_file | samtools sort -n -o - samtools_nsort_tmp | samtools fixmate /dev/stdin /dev/stdout | samtools sort -o - samtools_csort_tmp | samtools fillmd -u - $reference_fasta > $fixed_bam_file\tPP:bwa_sam

## Get ReadGroups

In [57]:
readgroup = client.getReadGroup(readGroupId)
print readgroup

ReadGroup({"info": {}, "updated": 1444697504850, "predictedInsertSize": 486, "stats": {"unalignedReadCount": -1, "alignedReadCount": -1, "baseCount": null}, "description": "SRP001293", "created": 1444697504850, "programs": [{"commandLine": "bwa index -a bwtsw $reference_fasta", "prevProgramId": null, "id": "bwa_index", "version": "0.5.9-r16", "name": "bwa"}, {"commandLine": "bwa aln -q 15 -f $sai_file $reference_fasta $fastq_file\tPP:bwa_index", "prevProgramId": null, "id": "bwa_aln_fastq", "version": "0.5.9-r16", "name": "bwa"}, {"commandLine": "bwa sampe -a 1458 -r $rg_line -f $sam_file $reference_fasta $sai_file(s) $fastq_file(s)\tPP:bwa_aln_fastq", "prevProgramId": null, "id": "bwa_sam", "version": "0.5.9-r16", "name": "bwa"}, {"commandLine": "samtools view -bSu $sam_file | samtools sort -n -o - samtools_nsort_tmp | samtools fixmate /dev/stdin /dev/stdout | samtools sort -o - samtools_csort_tmp | samtools fillmd -u - $reference_fasta > $fixed_bam_file\tPP:bwa_sam", "prevProgramId":

## Get Reads

In [58]:
results = client.searchReads([readGroupId], referenceId)
for i, result in enumerate(results):
    readId = result.id
print "%d reads in readgroup id = %s on chrom id = %s" % (i, readGroupId, referenceId)

997 reads in readgroup id = MWtnLXAzLXN1YnNldDpIRzAwNTM0OkVSUjAyMDIzOA== on chrom id = R1JDaDM4LXN1YnNldDoz


In [59]:
print result

ReadAlignment({"info": {"MD": ["15A75"], "NM": ["1"], "AM": ["37"], "RG": ["ERR020238"], "MQ": ["60"], "BQ": ["@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@"], "SM": ["37"], "X0": ["1"], "X1": ["0"], "XT": ["U"]}, "duplicateFragment": false, "readGroupId": "MWtnLXAzLXN1YnNldDpIRzAwNTM0OkVSUjAyMDIzOA==", "alignedQuality": [35, 37, 40, 39, 39, 41, 41, 39, 42, 43, 42, 42, 44, 43, 41, 44, 40, 42, 37, 43, 42, 40, 41, 41, 41, 43, 41, 42, 38, 43, 41, 41, 44, 44, 42, 44, 39, 42, 42, 44, 43, 43, 42, 45, 42, 43, 43, 42, 42, 43, 42, 42, 39, 44, 44, 44, 40, 44, 43, 42, 42, 43, 42, 44, 43, 42, 43, 44, 41, 41, 43, 42, 41, 41, 41, 42, 41, 40, 41, 42, 41, 40, 40, 40, 39, 41, 38, 38, 37, 37, 35], "failedVendorQualityChecks": false, "fragmentName": "ERR020238.14295398", "readNumber": 0, "properPlacement": true, "fragmentId": "TODO", "supplementaryAlignment": false, "numberReads": 2, "fragmentLength": -477, "secondaryAlignment": false, "alignedSequence": "CAA